## Загрузка библиотек 


In [1]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import glob
import copy
import neptune
#import splitfolders # Библиотека для разделения файлов картинок на train test
import matplotlib.gridspec as gridspec
cudnn.benchmark = True
plt.ion()


# Загрузка данных 

In [2]:
# Разделить данные на трейн тест не руками а библиотекой 
#splitfolders.ratio(
#  "C:\data\dataset",
#   output='C:\data\dataset\split',
#   seed=42,
#    ratio=(.8, .1, 0.1),
#    group_prefix=None,
  #  move=False)
    
run = neptune.init_run(
    project="neas1231/Neas1231",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJmY2VjMGIzOS01ZjI1LTQ1MTItODQxYi0zMjIyOWIwYWI0MzIifQ==",
)
#Нормализация и  аугментация  данных

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'E:\datasets\dataset'
run["config/data_dir"] = data_dir

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
              for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=10,
                                             shuffle=True, num_workers=8)
              for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) 
              for x in ['train', 'val']}

class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

D:\hroneniyas\anacondas\envs\gputorch\lib\site-packages\neptune\common\warnings.py:62: NeptuneWarning: To avoid unintended consumption of logging hours during interactive sessions, the following monitoring options are disabled unless set to 'True' when initializing the run: 'capture_stdout', 'capture_stderr', and 'capture_hardware_metrics'.
  warnings.warn(


https://app.neptune.ai/neas1231/Neas1231/e/NEAS-22


In [ ]:
#функция  для обучения и для высчитывания градиентов 

In [7]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=14):
    since = time.time()
    #Сохраняем  лучшие веса 
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)


        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  
            else:
                model.eval()   

            running_loss = 0.0
            running_corrects = 0

            # перенос  вычислений на куду 
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                
                optimizer.zero_grad()

                #
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    acc = (torch.sum(preds == labels.data)) / len(inputs)
                    run["logs/training/batch/loss"].append(loss)
                    run["logs/training/batch/acc"].append(acc)

                    # считаем градиенты тольок если трейн данные 
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # стасистика  
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # сохраняем не только лучшие веса , но и акураси 
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    model.load_state_dict(best_model_wts)
    return model

# Параметры для модели 

In [8]:
model_ft = models.efficientnet_b4(pretrained=True) 
num_ftrs = model_ft.classifier[1].in_features
# num_ftrs = model_ft.fc.in_features Для рес нета , но у эфинета другая архитектура 

model_ft.fc = nn.Linear(num_ftrs, len(class_names))
# cuda
model_ft = model_ft.to(device)
run["config/model"] = model_ft

criterion = nn.CrossEntropyLoss()
run["config/criterion"] =criterion

# оптимайзер  с лернинг рейт 
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)
run["config/optimizer"] = optimizer_ft
run["parameters"] = model_ft.parameters()

# редактируем лернинг рейт каждые 7 шагов 
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [9]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=14)

Epoch 0/13
----------
train Loss: 0.9840 Acc: 0.6970
val Loss: 0.4028 Acc: 0.8735

Epoch 1/13
----------
train Loss: 0.6006 Acc: 0.7963
val Loss: 0.3329 Acc: 0.8929

Epoch 2/13
----------
train Loss: 0.5294 Acc: 0.8222
val Loss: 0.3327 Acc: 0.8957

Epoch 3/13
----------
train Loss: 0.4994 Acc: 0.8297
val Loss: 0.3170 Acc: 0.8975

Epoch 4/13
----------
train Loss: 0.4460 Acc: 0.8464
val Loss: 0.3684 Acc: 0.8864

Epoch 5/13
----------
train Loss: 0.4406 Acc: 0.8536
val Loss: 0.3114 Acc: 0.8994

Epoch 6/13
----------
train Loss: 0.4182 Acc: 0.8557
val Loss: 0.3014 Acc: 0.8975

Epoch 7/13
----------
train Loss: 0.3278 Acc: 0.8876
val Loss: 0.2681 Acc: 0.9141

Epoch 8/13
----------
train Loss: 0.2814 Acc: 0.9018
val Loss: 0.2514 Acc: 0.9178

Epoch 9/13
----------
train Loss: 0.2518 Acc: 0.9149
val Loss: 0.2451 Acc: 0.9243

Epoch 10/13
----------
train Loss: 0.2488 Acc: 0.9136
val Loss: 0.2351 Acc: 0.9215

Epoch 11/13
----------
train Loss: 0.2266 Acc: 0.9238
val Loss: 0.2526 Acc: 0.9206

Ep

In [11]:
#Сохраняем модель
torch.save(model_ft,'efinet-0.925208acc-0.2546loss.pth')
torch.save(model_ft.state_dict(), "weights-efinet-0.925208acc-0.2546loss.pth")

In [ ]:
#Чистка кеша видеокарты
torch.cuda.memory_summary(device=None, abbreviated=False)
import gc
gc.collect()
import torch
torch.cuda.empty_cache()

In [10]:
run.stop()

Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 20 operations to synchronize with Neptune. Do not kill this process.
All 20 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/neas1231/Neas1231/e/NEAS-22/metadata
